In [1]:

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
def bar(x : torch.Tensor):
    if torch.mean(x) < 0:
        t = -0.125*x
    else:
        t = 1/2 * x ** 2
    return torch.norm(torch.sin(t)*t)

x0 = torch.tensor([2.2, -1.1])
bar(x0)


tensor(1.6352)

In [3]:
tsbar = torch.jit.script(bar)
print(tsbar.code)

def bar(x: Tensor) -> Tensor:
  _0 = torch.lt(torch.mean(x, dtype=None), 0)
  if bool(_0):
    t = torch.mul(x, -0.125)
  else:
    t = torch.mul(torch.pow(x, 2), 0.5)
  _1 = __torch__.torch.functional.norm(torch.mul(torch.sin(t), t), "fro", None, False, None, None, )
  return _1



In [4]:
x = torch.tensor(x0, requires_grad=True)

# Old "tagged"
y = bar(x)
dy = torch.autograd.grad(y, x)
dy

(tensor([-2.4868, -0.2468]),)

In [8]:
# new functional API

dbar = torch.autograd.functional.jacobian(tsbar, x, create_graph=True)
dbar


/home/awf/miniconda3/envs/onnx/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


tensor([-2.4868, -0.2468], grad_fn=<ViewBackward>)

In [21]:
# Print grad graph -- no conditionals.

dbar.grad_fn
var = dbar

seen = set()
output_nodes = (var.grad_fn,) if not isinstance(var, tuple) else tuple(v.grad_fn for v in var)

global_id = 0
ids = {}
def sid(v):
    global global_id
    global ids
    if id(v) not in ids:
        global_id+=1
        ids[id(v)]=global_id
    return ids[id(v)]

global c
def add_nodes(indent, var):
    tab = '  ' * (1+indent)

    if var in seen:
        print("Saw", id(var), "again")
        return

    if torch.is_tensor(var):
        print(tab, "T", sid(var), var.size())
    elif hasattr(var, 'variable'):
        u = var.variable
        print(tab, "V", u, sid(var), u.size())
    elif var in output_nodes:
        print(tab, "O", sid(var), str(type(var).__name__))
    else:
        if 'CppFunction' in str(var):
            global c
            c=var
        print(tab, "N", sid(var), str(type(var).__name__))
    seen.add(var)
    if hasattr(var, 'next_functions'):
        for u in var.next_functions:
            if u[0] is not None:
                print(tab, "E", sid(u), str(sid(var)))
                add_nodes(indent+1,u[0])
    if hasattr(var, 'saved_tensors'):
        for t in var.saved_tensors:
            print(tab, "E:", str(sid(t)), str(sid(var)))
            add_nodes(indent+1,t)

add_nodes(0, var.grad_fn)

   O 139902033161040 ViewBackward
   E 1 2
     N 3 StackBackward
     E 4 3
       N 5 MulBackward0
       E 6 5
         N 7 MulBackward1
         E 8 7
           N 9 MulBackward1
           E 10 9
             N 11 CppFunction
             E 12 11
               N 13 MulBackward1
               E 14 13
                 N 15 PowBackward0
                 E 16 15
                   N 17 ViewBackward
                   E 18 17
                     V tensor([ 2.2000, -1.1000], requires_grad=True) 19 torch.Size([2])
             E 20 11
               N 21 MulBackward0
               E 22 21
                 N 23 CppFunction
                 E 24 23
Saw 139902033189376 again
               E 25 21
                 N 26 MaskedFillBackward0
                 E 18 26
                   N 27 DivBackward0
                   E 28 27
                     N 29 NormBackward0
                     E 30 29
Saw 139901715391104 again
             E 31 11
Saw 139901715391104 again
       E 32 5
       

In [6]:
os.environ['KSC_PATH'] = '../../build/bin/ksc'
from ts2ks import ts2mod
def f(x: float) -> float:
    return torch.sin(x)*x
ks_bar = ts2mod(f)


[(<ksc.expr.Var object at 0x7f3d681be3a0>, <ksc.expr.Call object at 0x7f3d681be400>), (<ksc.expr.Var object at 0x7f3d681be460>, <ksc.expr.Call object at 0x7f3d681be580>)]
files /tmp/tmpqnfsz6d7.ks /tmp/tmpjk9h79__.kso /tmp/tmpa4vzm342.cpp
ks_str=
(def f Float ((_x_1 : Float))
  (let (_2 (aten::sin _x_1))
  (let (_4 (aten::mul _2 _x_1))
  _4)))

ksc: src/runtime/prelude.ks: openFile: does not exist (No such file or directory)



CalledProcessError: Command '['../../build/bin/ksc', '--generate-cpp', '--ks-source-file', 'src/runtime/prelude.ks', '--ks-source-file', 'src/runtime/prelude-aten.ks', '--ks-source-file', '/tmp/tmpqnfsz6d7.ks', '--ks-output-file', '/tmp/tmpjk9h79__.kso', '--cpp-output-file', '/tmp/tmpa4vzm342.cpp']' returned non-zero exit status 1.

In [54]:
1

/home/awf/miniconda3/envs/onnx/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1